In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches
import logging

# Configuration constants
FIGURE_SIZE = (5, 4)
THUMBNAIL_SIZE = (120, 120)
POSITION_MAP = {
    "top-left": (0, 0),
    "top-right": (0, 1),
    "bottom-left": (1, 0),
    "bottom-right": (1, 1)
}

# Setup logging
logging.basicConfig(level=logging.INFO)

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Widget Manager Class
class WidgetManager:
    def __init__(self, df):
        self.plot_type_widget = widgets.Dropdown(options=[], value=None, description="Plot Type:", layout=widgets.Layout(width='100%'))
        self.udl_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('UDL').unique(),
            value=[df.columns.get_level_values('UDL').unique()[0]],
            description='UDL:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.matu_widget = widgets.Dropdown(
            options=df.columns.get_level_values('Matu').unique(),
            value=df.columns.get_level_values('Matu').unique()[0],
            description='Matu:',
            layout=widgets.Layout(height='40px', width='100%')
        )
        self.param_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Param').unique(),
            value=[df.columns.get_level_values('Param').unique()[0]],
            description='Param:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.level_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Level').unique(),
            value=[df.columns.get_level_values('Level').unique()[0]],
            description='Level:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:', layout=widgets.Layout(height='40px', width='100%'))
        self.start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.plot_button = widgets.Button(description='Preview Plot', button_style='success', layout=widgets.Layout(width='28%', min_width='120px'))
        self.add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True, layout=widgets.Layout(width='28%', min_width='120px'))
        self.export_button = widgets.Button(description="Export to PPT", button_style='warning', layout=widgets.Layout(width='28%', min_width='120px'))
        self.position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:", layout=widgets.Layout(height='40px', width='100%'))
        self.add_slide_button = widgets.Button(description="Add Slide", button_style="primary", layout=widgets.Layout(width='28%', min_width='120px'))

# Step 3: Plot Manager to Register Plots and Manage Widgets
class PlotManager:
    def __init__(self):
        self.plots = {}

    def register_plot(self, name, plot_function, required_widgets):
        """Register a new plot type."""
        self.plots[name] = {
            "function": plot_function,
            "widgets": required_widgets
        }

    def get_plot_function(self, name):
        return self.plots.get(name, {}).get("function")

    def get_required_widgets(self, name):
        return self.plots.get(name, {}).get("widgets", [])

# Step 4: Registering Plot Functions
def create_time_series_plot(filtered_df, window):
    result_df = filtered_df.rolling(window=window).mean().fillna(0)
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    for column in result_df.columns:
        ax.plot(result_df.index, result_df[column], label=str(column))
    ax.set_title("Time Series Plot Preview")
    ax.legend()
    return fig

def create_stats_chart(filtered_df):
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    categories = filtered_df.columns.get_level_values('UDL').unique()
    min_values = filtered_df.min().values
    max_values = filtered_df.max().values
    avg_values = filtered_df.mean().values
    last_values = filtered_df.iloc[-1].values
    percentile_20 = filtered_df.quantile(0.2).values
    percentile_80 = filtered_df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    return fig

# Step 5: Utility function to generate plot image
def generate_plot_image(fig, ax):
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_stream.seek(0)
    img = Image.open(img_stream)
    ax.imshow(img, aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])

# Step 6: Slide management class
class SlideManager:
    def __init__(self):
        self.slides = [{"plots": []}]
        self.selected_slide_index = 0

    def add_slide(self):
        self.slides.append({"plots": []})
        self.selected_slide_index = len(self.slides) - 1

    def add_plot_to_slide(self, plot_data, position):
        slide_index = self.selected_slide_index
        # Remove any existing plot in the selected position
        self.slides[slide_index]["plots"] = [plot for plot in self.slides[slide_index]["plots"] if plot["position"] != position]
        # Add the new plot to the selected position
        self.slides[slide_index]["plots"].append(plot_data)

    def get_current_slide(self):
        return self.slides[self.selected_slide_index]

# Step 7: Main app class
class App:
    def __init__(self, df):
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.register_plots()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}
        self.output_plot = widgets.Output()
        self.main_display = widgets.Output(layout=widgets.Layout(width="72%", height="500px", border="1px solid black", padding="10px", margin="0 auto"))
        self.left_sidebar = widgets.VBox(layout=widgets.Layout(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0"))
        self.build_layout()
        self.bind_events()
        self.update_sidebar()
        self.run_app()

    def run_app(self):
        display(self)

    def register_plots(self):
        self.plot_manager.register_plot("Time Series Plot", create_time_series_plot, ["udl_widget", "matu_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"])
        self.plot_manager.register_plot("Custom Stats Chart", create_stats_chart, ["udl_widget", "param_widget", "level_widget", "start_date_widget", "end_date_widget"])

        # Update plot_type_widget with available plot types
        self.widgets.plot_type_widget.options = list(self.plot_manager.plots.keys())
        self.widgets.plot_type_widget.value = list(self.plot_manager.plots.keys())[0]  # Set default value

    def build_layout(self):
        control_buttons = widgets.GridBox([
            self.widgets.add_slide_button, self.widgets.export_button,
            self.widgets.plot_button, self.widgets.add_to_slide_button
        ], layout=widgets.Layout(grid_template_columns="repeat(2, 48%)", grid_gap="5px", width="100%", min_height="60px"))
        
        self.customization_window = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"),
            self.widgets.plot_type_widget,
            self.widgets.udl_widget,
            self.widgets.matu_widget,
            self.widgets.param_widget,
            self.widgets.level_widget,
            self.widgets.window_widget,
            self.widgets.start_date_widget,
            self.widgets.end_date_widget,
            self.widgets.position_dropdown,
            control_buttons,
        ], layout=widgets.Layout(width="30%", padding="15px", border="1px solid #d3d3d3", background_color="#f9f9f9"))
        
        # Display customization window on the right, plot or slide view in the center
        display(widgets.HBox([self.left_sidebar, self.main_display, self.customization_window]))

    def bind_events(self):
        self.widgets.plot_type_widget.observe(self.on_plot_type_change, names='value')
        self.widgets.plot_button.on_click(self.on_plot_button_clicked)
        self.widgets.add_to_slide_button.on_click(self.on_add_to_slide_button_clicked)
        self.widgets.add_slide_button.on_click(self.on_add_slide_button_clicked)
        self.widgets.export_button.on_click(self.on_export_button_clicked)

    def on_plot_type_change(self, change):
        selected_plot = change['new']
        required_widgets = self.plot_manager.get_required_widgets(selected_plot)

        for widget_name in ["udl_widget", "matu_widget", "param_widget", "level_widget", "window_widget", "start_date_widget", "end_date_widget"]:
            widget = getattr(self.widgets, widget_name)
            if widget_name in required_widgets:
                widget.layout.display = 'flex'
                widget.disabled = False
            else:
                widget.layout.display = 'none'
                widget.disabled = True

    def filter_data(self, df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date):
        filtered_df = df.loc[start_date:end_date, \
                             (df.columns.get_level_values('UDL').isin(selected_udl)) &
                             (df.columns.get_level_values('Matu').isin(selected_matu)) &
                             (df.columns.get_level_values('Param').isin(selected_param)) &
                             (df.columns.get_level_values('Level').isin(selected_level))]
        return filtered_df

    def on_plot_button_clicked(self, b):
        plot_type = self.widgets.plot_type_widget.value
        plot_function = self.plot_manager.get_plot_function(plot_type)

        # Handle None plot function (invalid plot type)
        if plot_function is None:
            with self.output_plot:
                clear_output(wait=True)
                print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
            return

        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = [self.widgets.matu_widget.value] if isinstance(self.widgets.matu_widget, widgets.Dropdown) else list(self.widgets.matu_widget.value)
        selected_param = list(self.widgets.param_widget.value)
        selected_level = list(self.widgets.level_widget.value)
        window = self.widgets.window_widget.value
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        if start_date > end_date:
            with self.output_plot:
                clear_output(wait=True)
                print("Start Date must be before End Date. Please correct your selection.")
                return

        filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date)
        if filtered_df.empty:
            with self.output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                self.widgets.add_to_slide_button.disabled = True
                return

        with self.main_display:
            clear_output(wait=True)
            try:
                fig = plot_function(filtered_df, window) if plot_type == "Time Series Plot" else plot_function(filtered_df)
                plt.show()
                self.current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": plot_type,
                    "position": self.widgets.position_dropdown.value
                }
                self.widgets.add_to_slide_button.disabled = False
            except Exception as e:
                print(f"An error occurred while generating the plot: {e}")

    def on_add_to_slide_button_clicked(self, b):
        if not self.current_plot_data:
            return
        
        # Re-fetch the currently selected position to ensure the latest value is used
        position = self.widgets.position_dropdown.value
        self.current_plot_data["position"] = position
        
        slide = self.slide_manager.get_current_slide()
        
        # Validate position is not already occupied
        if any(plot["position"] == position for plot in slide["plots"]):
            with self.main_display:
                clear_output(wait=True)
                print(f"Position '{position}' is already occupied. Please choose another position.")
                return

        # Add the plot to the slide if position is available
        self.slide_manager.add_plot_to_slide(self.current_plot_data.copy(), position)
        self.current_plot_data = {}  # Clear the current plot data to prevent duplicate additions
        self.switch_to_slide_view()
        self.update_sidebar()

    def on_add_slide_button_clicked(self, b):
        self.slide_manager.add_slide()
        self.update_sidebar()
        self.switch_to_slide_view()

    def on_export_button_clicked(self, b):
        prs = Presentation()
        for slide in self.slide_manager.slides:
            slide_layout = prs.slide_layouts[5]
            slide_to_add = prs.slides.add_slide(slide_layout)

            for plot_data in slide["plots"]:
                img_stream = BytesIO()
                fig = plot_data["figure"]
                fig.savefig(img_stream, format='png')
                img_stream.seek(0)
                left, top = self.get_slide_position(plot_data["position"])
                slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
                plt.close(fig)

        prs.save("Generated_Presentation.pptx")
        logging.info("Presentation exported as 'Generated_Presentation.pptx'")

    def get_slide_position(self, position):
        if position == "top-left":
            return Inches(0), Inches(0)
        elif position == "top-right":
            return Inches(5), Inches(0)
        elif position == "bottom-left":
            return Inches(0), Inches(3.75)
        elif position == "bottom-right":
            return Inches(5), Inches(3.75)
        return Inches(0), Inches(0)

    def switch_to_slide_view(self):
        self.main_display.clear_output(wait=True)
        with self.main_display:
            slide = self.slide_manager.get_current_slide()
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)
                    ax.set_title(plot_data["type"])
                else:
                    logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

            plt.tight_layout()
            plt.show()

    def update_sidebar(self):
        sidebar_content = []
        for i, slide in enumerate(self.slide_manager.slides):
            fig, axs = plt.subplots(2, 2, figsize=(4, 4))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)

            plt.tight_layout()

            buf = BytesIO()
            fig.savefig(buf, format="png")
            plt.close(fig)
            buf.seek(0)
            img = Image.open(buf)
            img.thumbnail(THUMBNAIL_SIZE)

            with BytesIO() as output:
                img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
            slide_button_box.on_click(lambda b, idx=i: self.select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        self.left_sidebar.children = sidebar_content

    def select_slide(self, index):
        self.slide_manager.selected_slide_index = index
        self.switch_to_slide_view()

# Step 8: Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
app = App(df)
display(app)

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches
import logging
import os

# Configuration constants
FIGURE_SIZE = (5, 4)
THUMBNAIL_SIZE = (120, 120)
POSITION_MAP = {
    "top-left": (0, 0),
    "top-right": (0, 1),
    "bottom-left": (1, 0),
    "bottom-right": (1, 1)
}

# Setup logging
logging.basicConfig(level=logging.INFO)

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Widget Manager Class
class WidgetManager:
    def __init__(self, df):
        self.plot_type_widget = widgets.Dropdown(options=[], value=None, description="Plot Type:", layout=widgets.Layout(width='100%'))
        self.udl_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('UDL').unique(),
            value=[df.columns.get_level_values('UDL').unique()[0]],
            description='UDL:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.matu_widget = widgets.Dropdown(
            options=df.columns.get_level_values('Matu').unique(),
            value=df.columns.get_level_values('Matu').unique()[0],
            description='Matu:',
            layout=widgets.Layout(height='40px', width='100%')
        )
        self.param_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Param').unique(),
            value=[df.columns.get_level_values('Param').unique()[0]],
            description='Param:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.level_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Level').unique(),
            value=[df.columns.get_level_values('Level').unique()[0]],
            description='Level:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:', layout=widgets.Layout(height='40px', width='100%'))
        self.start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.plot_button = widgets.Button(description='Preview Plot', button_style='success', layout=widgets.Layout(width='28%', min_width='120px'))
        self.add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True, layout=widgets.Layout(width='28%', min_width='120px'))
        self.export_button = widgets.Button(description="Export to PPT", button_style='warning', layout=widgets.Layout(width='28%', min_width='120px'))
        self.position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:", layout=widgets.Layout(height='40px', width='100%'))
        self.add_slide_button = widgets.Button(description="Add Slide", button_style="primary", layout=widgets.Layout(width='28%', min_width='120px'))

# Step 3: Plot Manager to Register Plots and Manage Widgets
class PlotManager:
    def __init__(self):
        self.plots = {}

    def register_plot(self, name, plot_function, required_widgets):
        """Register a new plot type."""
        self.plots[name] = {
            "function": plot_function,
            "widgets": required_widgets
        }

    def get_plot_function(self, name):
        return self.plots.get(name, {}).get("function")

    def get_required_widgets(self, name):
        return self.plots.get(name, {}).get("widgets", [])

# Step 4: Registering Plot Functions
def create_time_series_plot(filtered_df, window):
    result_df = filtered_df.rolling(window=window).mean().fillna(0)
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    for column in result_df.columns:
        ax.plot(result_df.index, result_df[column], label=str(column))
    ax.set_title("Time Series Plot Preview")
    ax.legend()
    return fig

def create_stats_chart(filtered_df):
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    categories = filtered_df.columns.get_level_values('UDL').unique()
    min_values = filtered_df.min().values
    max_values = filtered_df.max().values
    avg_values = filtered_df.mean().values
    last_values = filtered_df.iloc[-1].values
    percentile_20 = filtered_df.quantile(0.2).values
    percentile_80 = filtered_df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    return fig

# Step 5: Utility function to generate plot image
def generate_plot_image(fig, ax):
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_stream.seek(0)
    img = Image.open(img_stream)
    ax.imshow(img, aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])

# Step 6: Slide management class
class SlideManager:
    def __init__(self):
        self.slides = [{"plots": []}]
        self.selected_slide_index = 0

    def add_slide(self):
        self.slides.append({"plots": []})
        self.selected_slide_index = len(self.slides) - 1

    def add_plot_to_slide(self, plot_data, position):
        slide_index = self.selected_slide_index
        # Remove any existing plot in the selected position
        self.slides[slide_index]["plots"] = [plot for plot in self.slides[slide_index]["plots"] if plot["position"] != position]
        # Add the new plot to the selected position
        self.slides[slide_index]["plots"].append(plot_data)

    def get_current_slide(self):
        return self.slides[self.selected_slide_index]

# Step 7: Main app class
class App:
    def __init__(self, df):
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.register_plots()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}
        self.output_plot = widgets.Output()
        self.main_display = widgets.Output(layout=widgets.Layout(width="72%", height="500px", border="1px solid black", padding="10px", margin="0 auto"))
        self.left_sidebar = widgets.VBox(layout=widgets.Layout(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0"))
        self.build_layout()
        self.bind_events()
        self.update_sidebar()
        self.run_app()

    def run_app(self):
        display(self)

    def register_plots(self):
        self.plot_manager.register_plot("Time Series Plot", create_time_series_plot, ["udl_widget", "matu_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"])
        self.plot_manager.register_plot("Custom Stats Chart", create_stats_chart, ["udl_widget", "param_widget", "level_widget", "start_date_widget", "end_date_widget"])

        # Update plot_type_widget with available plot types
        self.widgets.plot_type_widget.options = list(self.plot_manager.plots.keys())
        self.widgets.plot_type_widget.value = list(self.plot_manager.plots.keys())[0]  # Set default value

    def build_layout(self):
        control_buttons = widgets.GridBox([
            self.widgets.add_slide_button, self.widgets.export_button,
            self.widgets.plot_button, self.widgets.add_to_slide_button
        ], layout=widgets.Layout(grid_template_columns="repeat(2, 48%)", grid_gap="5px", width="100%", min_height="60px"))
        
        self.customization_window = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"),
            self.widgets.plot_type_widget,
            self.widgets.udl_widget,
            self.widgets.matu_widget,
            self.widgets.param_widget,
            self.widgets.level_widget,
            self.widgets.window_widget,
            self.widgets.start_date_widget,
            self.widgets.end_date_widget,
            self.widgets.position_dropdown,
            control_buttons,
        ], layout=widgets.Layout(width="30%", padding="15px", border="1px solid #d3d3d3", background_color="#f9f9f9"))
        
        # Display customization window on the right, plot or slide view in the center
        display(widgets.HBox([self.left_sidebar, self.main_display, self.customization_window]))

    def bind_events(self):
        self.widgets.plot_type_widget.observe(self.on_plot_type_change, names='value')
        self.widgets.plot_button.on_click(self.on_plot_button_clicked)
        self.widgets.add_to_slide_button.on_click(self.on_add_to_slide_button_clicked)
        self.widgets.add_slide_button.on_click(self.on_add_slide_button_clicked)
        self.widgets.export_button.on_click(self.on_export_button_clicked)

    def on_plot_type_change(self, change):
        selected_plot = change['new']
        required_widgets = self.plot_manager.get_required_widgets(selected_plot)

        for widget_name in ["udl_widget", "matu_widget", "param_widget", "level_widget", "window_widget", "start_date_widget", "end_date_widget"]:
            widget = getattr(self.widgets, widget_name)
            if widget_name in required_widgets:
                widget.layout.display = 'flex'
                widget.disabled = False
            else:
                widget.layout.display = 'none'
                widget.disabled = True

    def filter_data(self, df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date):
        filtered_df = df.loc[start_date:end_date, \
                             (df.columns.get_level_values('UDL').isin(selected_udl)) &
                             (df.columns.get_level_values('Matu').isin(selected_matu)) &
                             (df.columns.get_level_values('Param').isin(selected_param)) &
                             (df.columns.get_level_values('Level').isin(selected_level))]
        return filtered_df

    def on_plot_button_clicked(self, b):
        plot_type = self.widgets.plot_type_widget.value
        plot_function = self.plot_manager.get_plot_function(plot_type)

        # Handle None plot function (invalid plot type)
        if plot_function is None:
            with self.output_plot:
                clear_output(wait=True)
                print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
            return

        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = [self.widgets.matu_widget.value] if isinstance(self.widgets.matu_widget, widgets.Dropdown) else list(self.widgets.matu_widget.value)
        selected_param = list(self.widgets.param_widget.value)
        selected_level = list(self.widgets.level_widget.value)
        window = self.widgets.window_widget.value
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        if start_date > end_date:
            with self.output_plot:
                clear_output(wait=True)
                print("Start Date must be before End Date. Please correct your selection.")
                return

        filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date)
        if filtered_df.empty:
            with self.output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                self.widgets.add_to_slide_button.disabled = True
                return

        with self.main_display:
            clear_output(wait=True)
            try:
                fig = plot_function(filtered_df, window) if plot_type == "Time Series Plot" else plot_function(filtered_df)
                plt.show()
                self.current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": plot_type,
                    "position": self.widgets.position_dropdown.value
                }
                self.widgets.add_to_slide_button.disabled = False
            except Exception as e:
                print(f"An error occurred while generating the plot: {e}")

    def on_add_to_slide_button_clicked(self, b):
        if not self.current_plot_data:
            return
        
        # Re-fetch the currently selected position to ensure the latest value is used
        position = self.widgets.position_dropdown.value
        self.current_plot_data["position"] = position
        
        slide = self.slide_manager.get_current_slide()
        
        # Validate position is not already occupied
        if any(plot["position"] == position for plot in slide["plots"]):
            with self.main_display:
                clear_output(wait=True)
                print(f"Position '{position}' is already occupied. Please choose another position.")
                return

        # Add the plot to the slide if position is available
        self.slide_manager.add_plot_to_slide(self.current_plot_data.copy(), position)
        self.current_plot_data = {}  # Clear the current plot data to prevent duplicate additions
        self.switch_to_slide_view()
        self.update_sidebar()

    def on_add_slide_button_clicked(self, b):
        self.slide_manager.add_slide()
        self.update_sidebar()
        self.switch_to_slide_view()

    def on_export_button_clicked(self, b):
        prs = Presentation()
        for slide in self.slide_manager.slides:
            slide_layout = prs.slide_layouts[5]
            slide_to_add = prs.slides.add_slide(slide_layout)

            for plot_data in slide["plots"]:
                img_stream = BytesIO()
                fig = plot_data["figure"]
                fig.savefig(img_stream, format='png')
                img_stream.seek(0)
                left, top = self.get_slide_position(plot_data["position"])
                slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
                plt.close(fig)

        pptx_filename = "Generated_Presentation.pptx"
        prs.save(pptx_filename)
        logging.info(f"Presentation exported as '{pptx_filename}'")
        
        # Ensure file visibility in Jupyter environment
        if os.path.exists(pptx_filename):
            print(f"Presentation saved successfully as '{pptx_filename}' in the current directory.")
        else:
            print("Error: The presentation could not be saved.")

    def get_slide_position(self, position):
        if position == "top-left":
            return Inches(0), Inches(0)
        elif position == "top-right":
            return Inches(5), Inches(0)
        elif position == "bottom-left":
            return Inches(0), Inches(3.75)
        elif position == "bottom-right":
            return Inches(5), Inches(3.75)
        return Inches(0), Inches(0)

    def switch_to_slide_view(self):
        self.main_display.clear_output(wait=True)
        with self.main_display:
            slide = self.slide_manager.get_current_slide()
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)
                    ax.set_title(plot_data["type"])
                else:
                    logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

            plt.tight_layout()
            plt.show()

    def update_sidebar(self):
        sidebar_content = []
        for i, slide in enumerate(self.slide_manager.slides):
            fig, axs = plt.subplots(2, 2, figsize=(4, 4))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)

            plt.tight_layout()

            buf = BytesIO()
            fig.savefig(buf, format="png")
            plt.close(fig)
            buf.seek(0)
            img = Image.open(buf)
            img.thumbnail(THUMBNAIL_SIZE)

            with BytesIO() as output:
                img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
            slide_button_box.on_click(lambda b, idx=i: self.select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        self.left_sidebar.children = sidebar_content

    def select_slide(self, index):
        self.slide_manager.selected_slide_index = index
        self.switch_to_slide_view()

# Step 8: Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
app = App(df)
display(app)
